<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [15]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [16]:
%%sql
SELECT title
FROM movies

 * sqlite:///movies.sqlite
Done.


title
The Shawshank Redemption
The Awakening
The Awakening
The Dark Knight
The Godfather


+ Show the performance dates for one of the movies.

In [5]:
%%sql
SELECT start_date, (SELECT title
             FROM movies
             WHERE IMDB_key LIKE 'tt0080402') AS title
FROM screenings
WHERE IMDB_key LIKE 'tt0080402'

 * sqlite:///movies.sqlite
Done.


start_date,title
2011-11-11,The Awakening


+ Show all data concerning performances at a given theatere
  on a given date.

In [6]:
%%sql
SELECT *
FROM screenings
WHERE start_date LIKE '2001-01-01'

 * sqlite:///movies.sqlite
Done.


start_time,start_date,n_seats,name,IMDB_key
13:15:00,2001-01-01,8,Filmstaden,tt0468569
15:15:15,2001-01-01,8,Kino,tt0111161


+ List all customers

In [7]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,password,full_name
Kalle15,Katt,Karl Erik
Hasse64,Hund,Hasse Björk
Knugen3,Silvia,Carl XVI Gustaf


+ List all tickets

In [17]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,username,name,start_time,start_date
3b510f0462c07c8be9dc82c18159522e,Kalle15,Kino,15:15:15,2001-01-01


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [18]:
%%sql
INSERT INTO tickets(username, name, start_time, start_date)
VALUES ('Knugen3', 'Kino', '15:15:15', '2001-01-01')

 * sqlite:///movies.sqlite
1 rows affected.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2021), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [19]:
%%sql
SELECT ticket_id
FROM tickets
WHERE username LIKE 'Knugen3'

 * sqlite:///movies.sqlite
Done.


ticket_id
ce01fcdb194da0488def1da367f46fd7


+ Try to insert two movie theaters with the same name (this
  should fail).

In [8]:
%%sql
INSERT INTO theaters(name, capacity)
VALUES
('kattbion', 523),
('kattbion', 333)


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.name
[SQL: INSERT INTO theaters(name, capacity)
VALUES
('kattbion', 523),
('kattbion', 333)]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [20]:
%%sql
INSERT INTO screenings(start_time, start_date, n_seats, name, IMDB_key)
VALUES ('08:15:00', '2011-11-11', 8, 'Körvabion', 'tt0080402')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: screenings.start_time, screenings.start_date
[SQL: INSERT INTO screenings(start_time, start_date, n_seats, name, IMDB_key)
VALUES ('08:15:00', '2011-11-11', 8, 'Körvabion', 'tt0080402')]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist (this should fail).

In [21]:
%%sql
INSERT INTO tickets(username, name, start_time, start_date)
VALUES ('Körvamannen', 'Kino', '15:15:15', '2001-01-01')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(username, name, start_time, start_date)
VALUES ('Körvamannen', 'Kino', '15:15:15', '2001-01-01')]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [8]:
%%sql
SELECT screening_id, screening_date, start_time, movie_title, movie_year, theater_name, (capacity-count(ticket_id))
FROM screenings
LEFT OUTER JOIN movies USING (imdb_id)
LEFT OUTER JOIN theaters USING (theater_name)
LEFT OUTER JOIN tickets USING (screening_id)
WHERE screening_id NOT NULL

 * sqlite:///movies.sqlite
Done.


screening_id,screening_date,start_time,movie_title,movie_year,theater_name,(capacity-count(ticket_id))
None,None,None,None,None,None,None
